# Contrastive Variational Autoencoder for the MNIST-Data-Set

Tobias Haase

I am slightly orienting myself on a paper from [Abid & Zou (2019)](https://arxiv.org/abs/1902.04601)
## Set Up
Firstly I am loading the required modules.

In [2]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
from torchvision.utils import save_image
import torchvision.datasets as datasets
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor

from torch.utils.data import Dataset, DataLoader
from torchvision.utils import make_grid
from torchvision.datasets import ImageFolder


import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
import imageio 

import os
os.chdir("/home/tchaase/Documents/Universitaet/Master-Arbeit/pytorch_mnist_VAE/")

As with the [vae](vae.ipynb) I will firstly also define the utility functions. 

In [3]:
def final_loss(bce_loss, z_mu, z_logvar, s_mu=None, s_logvar=None):
    """
    This function will add the reconstruction loss (BCELoss) and the KL-Divergence.
    KL-Divergence = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    :param bce_loss: reconstruction loss
    :param z_mu: mean from the latent vector of encoder_z
    :param z_logvar: log variance from the latent vector of encoder_z
    :param s_mu: mean from the latent vector of encoder_s (optional)
    :param s_logvar: log variance from the latent vector of encoder_s (optional)
    """
    BCE = bce_loss
    KLD_z = -0.5 * torch.sum(1 + z_logvar - z_mu.pow(2) - z_logvar.exp())
    if s_mu is not None and s_logvar is not None:
        KLD_s = -0.5 * torch.sum(1 + s_logvar - s_mu.pow(2) - s_logvar.exp())
        return BCE + KLD_z + KLD_s
    else:
        return BCE + KLD_z

Here the training function

In [4]:
def train(model, dataloader, dataset, device, optimizer, criterion):
    model.train()
    running_loss = 0.0
    counter = 0
    for i, (data, is_overlayed) in tqdm(enumerate(dataloader), total=len(dataset)):
        data = data.to(device)
        overlay_status = is_overlayed.to(device)

        optimizer.zero_grad()

        for j in range(data.size(0)):   #This seems weird, but I got errors related to the bce loss parameters being wrong, i.e. its both [1,32,32], but the input is [64,1,32,32, therefore I choose to iterate through participant and accumulate the gradient this way]
            single_data = data[j]
            single_overlay_status = overlay_status[j]

            if single_overlay_status:
                z_mean, z_log_var, s_mean, s_log_var, reconstructed_data = model(single_data)
                bce_loss = criterion(reconstructed_data, single_data)
                loss = final_loss(bce_loss, z_mean, z_log_var, s_mean, s_log_var)
            else:
                z_mean, z_log_var,reconstructed_data = model(single_data) #unsqueeze added to make the dimensions for the output comparable to the input)
                bce_loss = criterion(reconstructed_data, single_data)
                loss = final_loss(bce_loss, z_mean, z_log_var, None, None)  # Pass None for s_mean and s_log_var

            loss.backward()
            running_loss += loss.item()

        optimizer.step()

        counter += data.size(0)

    train_loss = running_loss / counter
    return train_loss

Here the validation functions:

In [5]:
def validate(model, dataloader, dataset, device, criterion):
    model.eval()
    running_loss = 0.0
    counter = 0
    with torch.no_grad():
        for i, data in tqdm(enumerate(dataloader), total=int(len(dataset)/dataloader.batch_size)):
            counter += 1
            data = data[0]
            data = data.to(device)
            reconstruction, z_mu, z_logvar, s_mu, s_logvar = model(data)
            bce_loss = criterion(reconstruction, data)
            loss = final_loss(bce_loss, z_mu, z_logvar, s_mu, s_logvar)
            running_loss += loss.item()
            
            # save the last batch input and output of every epoch
            if i == int(len(dataset)/dataloader.batch_size) - 1:
                recon_images = reconstruction
    val_loss = running_loss / counter
    return val_loss, recon_images


AttributeError: 'tuple' object has no attribute 'shape'

Now, lastly I will define some parameters that I will use before I define the full model.

In [6]:
kernel_size = 4 
init_channels = 8 # initial number of filters, first layers output. 
image_channels = 1 # MNIST images are grayscale
latent_dim = 16 # latent dimension for sampling
stride = 2
channels = 1  #working with grayscale. 
lr = 0.001
same = 0

In [39]:
import torch
import torch.nn as nn
import torch.nn.functional as F

    
class EncoderZ(nn.Module):
    def __init__(self, latent_dim):
        super(EncoderZ, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=image_channels, out_channels=init_channels, kernel_size=kernel_size, stride=stride, padding=1)
        self.conv2 = nn.Conv2d(in_channels=init_channels, out_channels=init_channels * 2, kernel_size=kernel_size, stride=stride, padding=1)
        self.fc1 = nn.Linear(init_channels*2* 8 * 8, 256) # t applies a linear transformation to the input data, which means it computes the dot product of the input and weight tensors, adds the bias term, and produces the output.
        self.fc_mean = nn.Linear(256, latent_dim)
        self.fc_log_var = nn.Linear(256, latent_dim)

    def forward(self, x):
        h = F.relu(self.conv1(x))
        h = F.relu(self.conv2(h))
        h_flat = h.view(h.size(0), -1)  # Flatten the tensor
        h = F.relu(self.fc1(h_flat))
        z_mean = self.fc_mean(h)
        z_log_var = self.fc_log_var(h)
        return z_mean, z_log_var

class EncoderS(nn.Module):
    def __init__(self, latent_dim):
        super(EncoderS, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=image_channels, out_channels=init_channels, kernel_size=kernel_size, stride=stride, padding=1)
        self.conv2 = nn.Conv2d(in_channels=init_channels, out_channels=init_channels * 2, kernel_size=kernel_size, stride=stride, padding=1)
        self.fc1 = nn.Linear(init_channels * 2*8 * 8, 256)
        self.fc_mean = nn.Linear(256, latent_dim)
        self.fc_log_var = nn.Linear(256, latent_dim)

    def forward(self, x):
        h = F.relu(self.conv1(x))
        h = F.relu(self.conv2(h))
        h_flat = h.view(h.size(0), -1)  # Flatten the tensor
        h = F.relu(self.fc1(h_flat))
        s_mean = self.fc_mean(h)
        s_log_var = self.fc_log_var(h)
        return s_mean, s_log_var

class Decoder(nn.Module):
    def __init__(self, latent_dim):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(latent_dim * 2, 256)
        self.fc2 = nn.Linear(256, 1024)  # Adjust the output size of fc2
        self.conv1 = nn.ConvTranspose2d(in_channels=init_channels, out_channels=init_channels*2, kernel_size=4, stride=2, padding=1)  # Set input channels to 1
        self.conv2 = nn.ConvTranspose2d(in_channels=init_channels*2, out_channels = image_channels, kernel_size=4, stride=2, padding= 1 )

    def forward(self, zs):
        x = F.relu(self.fc1(zs))
        x = F.relu(self.fc2(x))
        x = x.view(-1, 1, 16, 16)  # Reshape to a 4-dimensional tensor. I changed the first value to 1 recently, to have the input and output have equal dimensions.
        x = F.relu(self.conv1(x))
        x = torch.sigmoid(self.conv2(x))
        return x

class cVAE(nn.Module):
    def __init__(self, latent_dim):
        super(cVAE, self).__init__()
        self.encoder_z = EncoderZ(latent_dim)
        self.encoder_s = EncoderS(latent_dim)
        self.decoder = Decoder(latent_dim)
        self.overlay_status = None 

    def reparameterize(self, mean, log_var):
        std = torch.exp(0.5 * log_var)
        epsilon = torch.randn_like(std)
        return mean + epsilon * std
    
    def forward(self, x):
        z_mean, z_log_var = self.encoder_z(x)
        z = self.reparameterize(z_mean, z_log_var)
        
        if self.overlay_status: 
            mean, s_log_var = self.encoder_s(x)
            s = self.reparameterize(s_mean, s_log_var)
            zs = torch.cat([z, s], dim=1)
        else:
            empty_vector = torch.empty_like(z).detach()
            zs = torch.cat([z,empty_vector], dim = 1)
            
        reconstructed_data = self.decoder(zs)
        
        if self.overlay_status:
            return z_mean, z_log_var, s_mean, s_log_var, reconstructed_data
        else:
            return z_mean, z_log_var, reconstructed_data
        
    def get_overlay_status(self):
        return self.overlay_status

Right now, I can't directly access the images to superimpose them. Therefore, I will have to get the images with the following function. Additionally, I have incorporated a greyscaler into this. 

In [8]:
from PIL import Image
import pickle

def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo)
    return dict

def load_databatch(data_folder, idx, img_size=32):
    data_file = os.path.join(data_folder, 'train_data_batch_' + str(idx))
    d = unpickle(data_file)
    x = d['data']
    y = d['labels']
    mean_image = d['mean']

    x = x / np.float32(255)
    mean_image = mean_image / np.float32(255)

    # Labels are indexed from 1, shift it so that indexes start at 0
    y = [i - 1 for i in y]
    data_size = x.shape[0]

    x -= mean_image

    img_size2 = img_size * img_size

    x = np.dstack((x[:, :img_size2], x[:, img_size2:2 * img_size2], x[:, 2 * img_size2:]))
    x = x.reshape((x.shape[0], img_size, img_size, 3)).transpose(0, 3, 1, 2)

    # create mirrored images
    X_train = x[0:data_size, :, :, :]
    Y_train = y[0:data_size]
    X_train_flip = X_train[:, :, :, ::-1]
    Y_train_flip = Y_train
    X_train = np.concatenate((X_train, X_train_flip), axis=0)
    Y_train = np.concatenate((Y_train, Y_train_flip), axis=0)

    return dict(
        X_train=X_train.astype(np.float32),
        Y_train=np.array(Y_train, dtype=np.int32),
        mean=mean_image.astype(np.float32)
    )


Let's superimpose them with each other!

Firstly here is the way I loaded the mnist pictures as it was done before [here](./vae.ipynb). The data is loaded and resized to 32x32 to fit the Image-Net Data. 
I have once again split the data into a training and validation set. 


In [9]:
# Data transformations
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((32, 32)),
    torchvision.transforms.ToTensor(),
])

# Load the MNIST dataset
trainset = torchvision.datasets.MNIST(
    root='./input', train=True, download=True, transform=transform
)
testset = torchvision.datasets.MNIST(
    root='./input', train=False, download=True, transform=transform
)

Firstly I am unpickleing one batch and doing the calculations that were previously defined. 

In [10]:
batch_1 = load_databatch("./input/Imagenet32_train", 1)


Now, lets split the channels into matrices to then greyscale them. 

In [11]:
image_data = batch_1["X_train"]

# Get the number of images
num_images = len(image_data)

# Initialize an empty array to store the reshaped grayscale images
grayscale_images = np.empty((num_images, 32, 32), dtype=np.float32)

print("The shape is now: ",image_data.shape)

# Reshape the images
for i, image in enumerate(image_data):
    # Reshape the image to match the color channel dimensions (3x32x32)
    image = np.reshape(image, (3, 32, 32))

    # Split the image into separate color channels
    red_channel = image[0]
    green_channel = image[1]
    blue_channel = image[2]

    # Combine the color channels weighted by their respective coefficients to form the grayscale image - these values are taken from recommedations from ChatGPT
    grayscale_image = 0.2989 * red_channel + 0.5870 * green_channel + 0.1140 * blue_channel

    # Normalize the grayscale image to the range [0, 1]
    grayscale_image /= 255.0

    # Store the grayscale image in the array
    grayscale_images[i] = grayscale_image

# Store the reshaped grayscale images in a new entry in the dictionary
batch_1["X_train_gray"] = grayscale_images
print("The shape is now: ",grayscale_images.shape)

The shape is now:  (256232, 3, 32, 32)
The shape is now:  (256232, 32, 32)


This seemed to have worked. Now, lets move on to overlaying them. The following still needs proper testing. 

In [12]:
import random
import torch
from torchvision import datasets, transforms

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load MNIST dataset
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])
mnist_data = datasets.MNIST(root='./input', train=True, download=True, transform=transform)

# Access the background images from the dictionary
background_images = batch_1["X_train_gray"]

# Create a list to store the overlaid images
overlaid_images = []
overlayed_count = 0
not_overlayed_count = 0


# Work in progress, made two channels instead to actually get it to run but should remove channels. Sometimes in code two channels show up I dont understand why. 

for i in range(len(background_images)):
    background_image = background_images[i]
    if i < len(mnist_data) // 2:  # Overlay MNIST image for half of the background images
        random_index = random.choice([0, 1, 2])  # Choose random digit 0, 1, or 2
        mnist_image, _ = mnist_data[random_index]

        # Convert background image to tensor and move it to the CUDA device
        background_tensor = torch.tensor(background_image).unsqueeze(0).to(device)

        # Resize the background image to match the MNIST image dimensions
        resized_background_tensor = transforms.Resize((32, 32))(background_tensor)

        # Remove extra channel if present
        if resized_background_tensor.shape[0] == 2:
            resized_background_tensor = resized_background_tensor[0]

        # Resize MNIST image to match the background image dimensions
        mnist_resized = transforms.Resize((32, 32))(mnist_image)

        overlaid_tensor = torch.cat([resized_background_tensor, mnist_resized.to(device)], dim=0)
        overlaid_images.append((overlaid_tensor, True))  # Store information that the image was overlaid
        overlayed_count += 1
    else:
        background_tensor = torch.tensor(background_image).unsqueeze(0).to(device)

        # Remove extra channel if present
        if background_tensor.shape[0] == 2:
            background_tensor = background_tensor[0]

        overlaid_images.append((background_tensor, False))  # Store information that the image was not overlaid
        not_overlayed_count += 1
        


# Print the counts of overlaid and non-overlaid images
print(f"Overlayed Images: {overlayed_count}")
print(f"Not Overlayed Images: {not_overlayed_count}")

/home/tchaase/miniconda3/envs/pytorch11.7/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Overlayed Images: 30000
Not Overlayed Images: 226232


In [13]:
for background, is_overlayed in overlaid_images:
    if not torch.is_tensor(background):
        print("Non-tensor background found!")
        # Handle the case when a non-tensor background is encountered
    else:
        pass

Now here is the data-loader. 

In [14]:
class OverlaidDataset(Dataset):
    def __init__(self, overlaid_images, channel_index=0):
        self.overlaid_images = overlaid_images
        self.to_tensor = ToTensor()
        self.channel_index = channel_index

    def __len__(self):
        return len(self.overlaid_images)

    def __getitem__(self, index):
        image, is_overlayed = self.overlaid_images[index]

        # Select the specified channel
        image = image.narrow(0, self.channel_index, 1)

        return image, is_overlayed


# Create the overlaid dataset
overlaid_dataset = OverlaidDataset(overlaid_images)

# Create the dataloader
batch_size = 64
overlaid_dataloader = DataLoader(overlaid_dataset, batch_size=batch_size, shuffle=True)


Checking again that I still have the right data type. 

In [15]:
dataloader_images, _ = next(iter(overlaid_dataloader))

for idx, x in enumerate(dataloader_images):
    if not torch.is_tensor(x):
        print("Non-tensor image found! This is of the type:", type(x))
        # Handle the case when a non-tensor background is encountered
    else:
        # Process the background tensor
        # ...
        pass

Now, I am creating a training loop. 

In [40]:
model = cVAE(latent_dim=64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCELoss(reduction='sum')

num_epochs = 10
for epoch in range(num_epochs):
    # Train the model
    train_loss = train(model, overlaid_dataloader, overlaid_dataset, device, optimizer, criterion)
    
    # Validate the model
    val_loss, recon_images = validate(model, overlaid_dataloader, overlaid_dataset, device, criterion)
    
    # Print the losses
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")


  0%|          | 0/256232 [00:00<?, ?it/s]

  0%|          | 0/256232 [00:00<?, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (16x36 and 1024x256)

In [64]:
for _, is_overlayed in overlaid_dataset:
    if not isinstance(is_overlayed, bool):
        print(is_overlayed)
